# Neptune Result Fetching Notebook

In [ ]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3

# required to enable non-blocking interaction:
%gui qt5

# import ipywidgets as widgets
# widgets.Widget.control_comm_live = False

import sys
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from neuropy.utils.result_context import IdentifyingContext
from nptyping import NDArray
import neuropy.utils.type_aliases as types
from collections import defaultdict

import numpy as np
import pandas as pd
import re
from pathlib import Path
from datetime import datetime

from neuropy.utils.indexing_helpers import PandasHelpers
from pyphocorehelpers.indexing_helpers import partition_df
# Set the maximum number of columns to display
pd.set_option('display.max_columns', 100)

import IPython
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.notebook_helpers import NotebookCellExecutionLogger
from pyphocorehelpers.assertion_helpers import Assert

# Jupyter-lab enable printing for any line on its own (instead of just the last one in the cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# _notebook_path:Path = Path(IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())).resolve() # Finds the path of THIS notebook

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')

# Switch to the desired interactivity mode
plt.interactive(True)

import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
from pyphocorehelpers.plotting.media_output_helpers import fig_to_clipboard
from pyphocorehelpers.Filesystem.path_helpers import file_uri_from_path, sanitize_filename_for_Windows
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, simple_path_display_widget
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_helper_save_figures, _helper_build_figure, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results
from pyphocorehelpers.assertion_helpers import Assert

# from ..PendingNotebookCode import plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import find_csv_files, find_HDF5_files, find_most_recent_files
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_histograms, plot_stacked_histograms

from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DecoderDecodedEpochsResult
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import load_across_sessions_exported_files, _process_and_load_exported_file, _common_cleanup_operations

from pyphocorehelpers.programming_helpers import metadata_attributes
from pyphocorehelpers.function_helpers import function_attributes

from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import build_session_t_delta, _new_process_csv_files, _old_process_csv_files
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import get_nested_value, flatten_dict
# from neptune.types import File
from neptune.attributes.atoms.file import File

debug_print: bool = False

_TODAY_DAY_ONLY_DATE: str = "2024-10-28"
TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_Apogee"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_GL"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_Lab"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_rMBP"

print(f'TODAY_DAY_DATE: {TODAY_DAY_DATE}')

types.session_str: TypeAlias = str # a unique session identifier

import neptune # for logging progress and results
from neptune.types import File
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import PhoDibaProjectsNeptuner, Neptuner, AutoValueConvertingNeptuneRun, set_environment_variables, SessionDescriptorString, RunID, NeptuneRunCollectedResults, KnownNeptuneProjects

## Gets the notebook filepath for Neptune:
import IPython
from pyphocorehelpers.programming_helpers import IPythonHelpers
# notebook_filepath: str = IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())
# assert Path(notebook_filepath).resolve().exists(), f"found notebook filepath: '{notebook_filepath}' does not exist"
# notebook_filepath

from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import ProgrammaticDisplayFunctionTestingFolderImageLoading

combined_neptuner: PhoDibaProjectsNeptuner = PhoDibaProjectsNeptuner()


In [ ]:
# neptune_run_collected_results: NeptuneRunCollectedResults = neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-10-01', n_recent_results=3)
# neptune_run_collected_results: NeptuneRunCollectedResults = neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-07-20', n_recent_results=16)
# neptune_run_collected_results: NeptuneRunCollectedResults = neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-10-15', n_recent_results=16)
# neptune_run_collected_results: NeptuneRunCollectedResults = neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-09-20', n_recent_results=3)
# neptune_run_collected_results: NeptuneRunCollectedResults = neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-10-15', n_recent_results=16)
# combined_neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-10-15', n_recent_results=16)

combined_neptuner.get_most_recent_session_runs(utility_oldest_included_run_date='2024-10-28', utility_n_recent_results=2,
											   oldest_included_run_date='2024-10-20', n_recent_results=2)
# neptune_run_collected_results
# runs_dict, most_recent_runs_table_df = neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-06-01', n_recent_results=2)
# most_recent_runs_table_df

most_recent_runs_table_df_dict, csv_output_paths = combined_neptuner.save_run_history_CSVs(TODAY_DAY_DATE=TODAY_DAY_DATE)

##  2024-09-24 - tries to get the figures from the runs

In [ ]:
"outputs/figures/display_fn_name:display_short_long_pf1D_comparison/track:long"
"outputs/figures/display_fn_name:display_short_long_pf1D_comparison/track:short"

"display_fn_name:display_long_short_laps"

"display_fn_name:_display_grid_bin_bounds_validation"



# Particular Run: `https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-1343`
# most_recent_runs_context_indexed_run_extra_data
# a_session_descriptor_str: str = 'kdiba_gor01_two_2006-6-07_16-40-19'
# a_session_descriptor_str: str = 'kdiba_gor01_one_2006-6-09_1-22-43'
# a_run = neptune_run_collected_results.runs_dict['LS2023-1343'] # kdiba_gor01_two_2006-6-07_16-40-19_sess

# fig_input_key: str = "outputs/figures/display_fn_name:display_short_long_pf1D_comparison/track:long"
# fig_input_key: str = "display_fn_name:display_short_long_pf1D_comparison/track:long"
# fig_input_key: str = "display_fn_name:display_short_long_pf1D_comparison/track:short"
# download_image(a_run=a_run, fig_input_key=fig_input_key, a_session_descriptor_str=a_session_descriptor_str)


# Support Incomplete Keys to export multiple images: _________________________________________________________________ #
# fig_input_key: str = "display_fn_name:BatchPhoJonathanReplayFRC/plot_result_set:shared/page:1of6/aclus:(2,4,5,6,7,8,9,11,12,13)"
# fig_input_key: str = "display_fn_name:BatchPhoJonathanReplayFRC" # plot_result_set:shared/page:1of6/aclus:(2,4,5,6,7,8,9,11,12,13)
# check if the key ends with a forward-slash, in which case it may describe multiple images
# fig_input_key.endswith('/')

# {'plot_result_set:long_only': {'aclus:(48)': <neptune.attributes.atoms.file.File at 0x282be50c760>},
#  'plot_result_set:shared': {'page:1of3': {'aclus:(7,41,20,23,32,63,46,52,15,54,51,60,50,28,31,49,26,5,40,62)': <neptune.attributes.atoms.file.File at 0x282be50c7c0>},
#   'page:1of6': {'aclus:(2,4,5,6,7,8,9,11,12,13)': <neptune.attributes.atoms.file.File at 0x282be50caf0>},
#   'page:2of3': {'aclus:(11,55,27,45,24,53,17,56,6,35,34,8,25,21,39,33,44,64,43,2)': <neptune.attributes.atoms.file.File at 0x282be50cb20>},
#   'page:2of6': {'aclus:(14,15,16,17,20,21,22,23,24,25)': <neptune.attributes.atoms.file.File at 0x282be4f2040>},
#   'page:3of3': {'aclus:(4,9,12,13,14,16,22,29,30,36,37,38,42,47,57,58,59,61)': <neptune.attributes.atoms.file.File at 0x282be4df9d0>},
#   'page:3of6': {'aclus:(26,27,28,29,30,31,32,33,34,35)': <neptune.attributes.atoms.file.File at 0x282be4dfb80>},
#   'page:4of6': {'aclus:(36,37,38,39,40,41,42,43,44,45)': <neptune.attributes.atoms.file.File at 0x282be4dfc70>},
#   'page:5of6': {'aclus:(46,47,49,50,51,52,53,54,55,56)': <neptune.attributes.atoms.file.File at 0x282be4df8e0>},
#   'page:6of6': {'aclus:(57,58,59,60,61,62,63,64)': <neptune.attributes.atoms.file.File at 0x282be4df5b0>}},
#  'plot_result_set:short_only': {'aclus:(3,10,18,19,65)': <neptune.attributes.atoms.file.File at 0x282be4df760>}}


# {'plot_result_set:long_only/aclus:(48)': <neptune.attributes.atoms.file.File at 0x282be50c760>,
#  'plot_result_set:shared/page:1of3/aclus:(7,41,20,23,32,63,46,52,15,54,51,60,50,28,31,49,26,5,40,62)': <neptune.attributes.atoms.file.File at 0x282be50c7c0>,
#  'plot_result_set:shared/page:1of6/aclus:(2,4,5,6,7,8,9,11,12,13)': <neptune.attributes.atoms.file.File at 0x282be50caf0>,
#  'plot_result_set:shared/page:2of3/aclus:(11,55,27,45,24,53,17,56,6,35,34,8,25,21,39,33,44,64,43,2)': <neptune.attributes.atoms.file.File at 0x282be50cb20>,
#  'plot_result_set:shared/page:2of6/aclus:(14,15,16,17,20,21,22,23,24,25)': <neptune.attributes.atoms.file.File at 0x282be4f2040>,
#  'plot_result_set:shared/page:3of3/aclus:(4,9,12,13,14,16,22,29,30,36,37,38,42,47,57,58,59,61)': <neptune.attributes.atoms.file.File at 0x282be4df9d0>,
#  'plot_result_set:shared/page:3of6/aclus:(26,27,28,29,30,31,32,33,34,35)': <neptune.attributes.atoms.file.File at 0x282be4dfb80>,
#  'plot_result_set:shared/page:4of6/aclus:(36,37,38,39,40,41,42,43,44,45)': <neptune.attributes.atoms.file.File at 0x282be4dfc70>,
#  'plot_result_set:shared/page:5of6/aclus:(46,47,49,50,51,52,53,54,55,56)': <neptune.attributes.atoms.file.File at 0x282be4df8e0>,
#  'plot_result_set:shared/page:6of6/aclus:(57,58,59,60,61,62,63,64)': <neptune.attributes.atoms.file.File at 0x282be4df5b0>,
#  'plot_result_set:short_only/aclus:(3,10,18,19,65)': <neptune.attributes.atoms.file.File at 0x282be4df760>}


In [ ]:
from neuropy.utils.indexing_helpers import flatten_dict
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display, HTML, Javascript
from pyphocorehelpers.gui.Jupyter.simple_widgets import filesystem_path_folder_contents_widget, fullwidth_path_widget, simple_path_display_widget
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import flatten_context_nested_dict


# fig_input_key: str = "display_fn_name:display_long_short_laps"
# fig_input_key: str = "display_fn_name:display_short_long_pf1D_comparison/track:long"
# fig_input_key: str = "display_fn_name:display_short_long_pf1D_comparison/track:short"
# fig_input_key: str = "display_fn_name:running_and_replay_speeds_over_time"
# fig_input_key: str = "display_fn_name:running_and_replay_speeds_over_time"
fig_input_key: str = "display_fn_name:trial_to_trial_reliability"

# Support Incomplete Keys to export multiple images: _________________________________________________________________ #
# fig_input_key: str = "display_fn_name:BatchPhoJonathanReplayFRC/plot_result_set:shared/page:1of6/aclus:(2,4,5,6,7,8,9,11,12,13)"
# fig_input_key: str = "display_fn_name:BatchPhoJonathanReplayFRC" # plot_result_set:shared/page:1of6/aclus:(2,4,5,6,7,8,9,11,12,13)
# check if the key ends with a forward-slash, in which case it may describe multiple images
# fig_input_key.endswith('/')

# fig_input_key: str = "display_fn_name:plot_all_epoch_bins_marginal_predictions/subplot_name:Laps all_epoch_binned Marginals"

# fig_input_key: str = "display_fn_name:display_long_short_laps"
# fig_input_key: str = "display_fn_name:_display_grid_bin_bounds_validation"

_flattened_context_path_dict = combined_neptuner.try_get_figures(fig_input_key=fig_input_key)
_flattened_context_path_dict

In [ ]:
from pyphocorehelpers.gui.Jupyter.JupyterImageNavigatorWidget import ImageContextViewer

viewer = ImageContextViewer(_flattened_context_path_dict)
viewer.display()

# "pin01_one_fet11-01_12..." - bad looking position data
# "pin01_one_11-02_17_46" - same as above, identical looking


In [ ]:
## INPUTS: _final_out_dict_dict: Dict[ContextDescStr, Dict[ImageNameStr, Dict[datetime, Path]]]
from pyphocorehelpers.gui.Jupyter.JupyterImageNavigatorWidget import build_context_images_navigator_widget, ContextSidebar, ImageNavigator


context_tabs_dict = {curr_context_desc_str:build_context_images_navigator_widget(curr_context_images_dict, curr_context_desc_str=curr_context_desc_str, max_num_widget_debug=2) for curr_context_desc_str, curr_context_images_list in list(_flattened_context_path_dict.items())}
sidebar = ContextSidebar(context_tabs_dict)
sidebar.display()



In [ ]:
## INPUTS: _context_figures_dict

# {'kdiba_gor01_two_2006-6-07_16-40-19/LS2023-1370/gor01_two_2006-6-07_16-40-19/display_fn_name:_display_grid_bin_bounds_validation': 'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/neptune/commander.pho/PhoDibaLongShortUpdated/figs/gor01_two_2006-6-07_16-40-19/_display_grid_bin_bounds_validation.png',
#  'kdiba_gor01_two_2006-6-07_16-40-19/LS2023-1354/gor01_two_2006-6-07_16-40-19/display_fn_name:_display_grid_bin_bounds_validation': 'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/neptune/commander.pho/PhoDibaLongShortUpdated/figs/gor01_two_2006-6-07_16-40-19/_display_grid_bin_bounds_validation.png',
#  'kdiba_gor01_one_2006-6-12_15-55-31/LS2023-1369/gor01_one_2006-6-12_15-55-31/display_fn_name:_display_grid_bin_bounds_validation': 'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/neptune/commander.pho/PhoDibaLongShortUpdated/figs/gor01_one_2006-6-12_15-55-31/_display_grid_bin_bounds_validation.png',
#  'kdiba_gor01_one_2006-6-09_1-22-43/LS2023-1368/gor01_one_2006-6-09_1-22-43/display_fn_name:_display_grid_bin_bounds_validation': 'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/neptune/commander.pho/PhoDibaLongShortUpdated/figs/gor01_one_2006-6-09_1-22-43/_display_grid_bin_bounds_validation.png',
#  'kdiba_gor01_one_2006-6-08_14-26-15/LS2023-1367/gor01_one_2006-6-08_14-26-15/display_fn_name:_display_grid_bin_bounds_validation': 'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/neptune/commander.pho/PhoDibaLongShortUpdated/figs/gor01_one_2006-6-08_14-26-15/_display_grid_bin_bounds_validation.png',
#  'kdiba_gor01_one_2006-6-08_14-26-15/LS2023-1361/gor01_one_2006-6-08_14-26-15/display_fn_name:_display_grid_bin_bounds_validation': 'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/neptune/commander.pho/PhoDibaLongShortUpdated/figs/gor01_one_2006-6-08_14-26-15/_display_grid_bin_bounds_validation.png',
#  'kdiba_gor01_one_2006-6-07_11-26-53/LS2023-1351/gor01_one_2006-6-07_11-26-53/display_fn_name:_display_grid_bin_bounds_validation': 'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/neptune/commander.pho/PhoDibaLongShortUpdated/figs/gor01_one_2006-6-07_11-26-53/_display_grid_bin_bounds_validation.png'}


# Parse ProgrammaticDisplayFunctionTesting

In [ ]:
import os
from datetime import datetime
from pathlib import Path
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import ProgrammaticDisplayFunctionTestingFolderImageLoading

# programmatic_display_function_testing_path: Path = Path(r"C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting").resolve()
programmatic_display_function_testing_path: Path = Path('/home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting').resolve()
programmatic_display_function_outputs_df, programmatic_display_function_outputs_tuples, csv_out_path = ProgrammaticDisplayFunctionTestingFolderImageLoading.parse_ProgrammaticDisplayFunctionTesting_image_folder(programmatic_display_function_testing_path=programmatic_display_function_testing_path)
programmatic_display_function_outputs_df
csv_out_path


In [ ]:
_flattened_context_path_dict

In [ ]:
# file_uri_from_path()


# Run Structure/Variables

In [ ]:
from pyphocorehelpers.print_helpers import DocumentationFilePrinter, TypePrintMode, print_keys_if_possible
# doc_output_parent_folder: Path = Path('EXTERNAL/DEVELOPER_NOTES/DataStructureDocumentation').resolve() # ../.
doc_output_parent_folder: Path = Path(r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/EXTERNAL/DEVELOPER_NOTES/DataStructureDocumentation').resolve() # ../.
print(f"doc_output_parent_folder: {doc_output_parent_folder}")
assert doc_output_parent_folder.exists()


## Document `compute_and_export_session_alternative_replay_wcorr_shuffles_completion_function_results`
doc_printer = DocumentationFilePrinter(doc_output_parent_folder=doc_output_parent_folder, doc_name='_context_run_structure_dict')
doc_printer.save_documentation('_context_run_structure_dict', _context_run_structure_dict)

In [ ]:
run_structure_dict_dict = combined_neptuner.projects_resolved_run_structure_dict

figures_results = []
logs_results = []

# for a_name, a_run_structure_dict in run_structure_dict_dict.items():

figures_run_structure_dict = run_structure_dict_dict['main']
for a_ctxt, a_dict in figures_run_structure_dict.items():
	a_dict
	# figures_results.append(a_dict['outputs']['figures'])
# run_structure_dict_dict[a_name] = a_run_collected_results.get_resolved_structure()
	
figures_results

# Log Viewer Widget

In [ ]:
# _context_log_files_dict = neptune_run_collected_results.download_uploaded_log_files(neptune_logs_output_path=neptune_logs_output_path)
# _context_log_files_dict
combined_neptuner

In [ ]:
_context_log_files_dict, context_indexed_run_logs, _out_log_paths = combined_neptuner.try_get_run_logs()
_context_log_files_dict
context_indexed_run_logs
_out_log_paths

# 20241028T214208--PHDBATCH-2169--kdiba--gor01--two--2006-6-07_16-40-19--run.log


In [ ]:
most_recent_runs_table_df['final_log_file_path']


In [ ]:
_out_log_paths

In [ ]:
# Call the function to build and display the interactive session display
most_recent_runs_session_descriptor_string_to_context_map: Dict[SessionDescriptorString, IdentifyingContext] = combined_neptuner.utility_run_collected_results.most_recent_runs_session_descriptor_string_to_context_map
most_recent_runs_context_indexed_run_extra_data: Dict[IdentifyingContext, Dict] = combined_neptuner.utility_run_collected_results.most_recent_runs_context_indexed_run_extra_data
# most_recent_runs_context_indexed_run_extra_data # SessionTuple(format_name='kdiba', animal='pin01', exper_name='one', session_name='11-02_17-46-44', session_descriptor_string='kdiba_pin01_one_11-02_17-46-44_sess', id='LS2023-1335', hostname='gl3126.arc-ts.umich.edu', creation_time=Timestamp('2024-08-29 16:39:16.613000'), running_time=8735.629, ping_time=Timestamp('2024-09-24 08:38:06.626000'), monitoring_time=1543, size=28686905.0, tags='11-02_17-46-44,one,kdiba,pin01', entrypoint='figures_kdiba_pin01_one_11-02_17-46-44.py')

interactive_layout = Neptuner.build_interactive_session_run_logs_widget(context_indexed_run_logs, most_recent_runs_session_descriptor_string_to_context_map, most_recent_runs_context_indexed_run_extra_data)
display(interactive_layout)

In [ ]:
# does not trigger change:
# .on_node_selected(change: {'name': 'selected', 'old': True, 'new': False, 'owner': Node(name='11-02_17-46-44'), 'type': 'change'})
# .on_node_selected(change: {'name': 'selected', 'old': False, 'new': True, 'owner': Node(name='fet11-01_12-58-54', selected=True), 'type': 'change'})
# Selected context: {'format_name': 'kdiba', 'animal': 'pin01', 'exper_name': 'one', 'session_name': 'fet11-01_12-58-54'}


In [ ]:
empty_session_tuple = {
    'format_name': '',
    'animal': '',
    'exper_name': '',
    'session_name': '',
    'session_descriptor_string': '',
    'id': '<Selection Not Found>',
    'hostname': '',
    'creation_time': '',
    'running_time': '',
    'ping_time': '',
    'monitoring_time': '',
    'size': '',
    'tags': '',
    'entrypoint': ''
}

print(list(empty_session_tuple.keys())) # ['format_name', 'animal', 'exper_name', 'session_name', 'session_descriptor_string', 'id', 'hostname', 'creation_time', 'running_time', 'ping_time', 'monitoring_time', 'size', 'tags', 'entrypoint']
display_session_extra_data_keys = ['id', 'hostname', 'creation_time', 'running_time', 'ping_time', 'monitoring_time', 'size', 'tags', 'entrypoint']

# hostname: gl3126.arc-ts.umich.edu
# creation_time: 2024-08-29 16:47:55.150000
# running_time: 8512.876
# ping_time: 2024-09-24 08:38:06.349000
# monitoring_time: 1291
# size: 24744223.0
# tags: one,kdiba,11-02_19-28-0,pin01
# entrypoint: figures_kdiba_pin01_one_11-02_19-28-0.py



In [ ]:
from pandas import Timestamp
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.gui.Jupyter.TreeWidget import JupyterTreeWidget

## INPUTS: context_indexed_run_logs, most_recent_runs_session_descriptor_string_to_context_map, header_hbox

# Tree Widget ________________________________________________________________________________________________________ #
included_session_contexts: List[IdentifyingContext] = list(most_recent_runs_session_descriptor_string_to_context_map.values())
jupyter_tree_widget = JupyterTreeWidget(included_session_contexts=included_session_contexts,
                                        on_selection_changed_callbacks=[],
                                        display_on_init=False)
# type(jupyter_tree_widget.tree) # ipytree.tree.Tree

# Set a layout for the tree to prevent it from being cut off
jupyter_tree_widget.tree.layout = widgets.Layout(min_width='200px', max_width='600px', overflow='auto', height='auto')

# Content Widget _____________________________________________________________________________________________________ #
def build_session_tuple_header_widget(a_session_tuple: Tuple):
    # Create widgets for each key-value pair
    # widgets_list = [widgets.Label(f"{key}: {value}") for key, value in a_session_tuple.items()]
    
    # Create a dictionary to hold the label widgets
    header_label_widgets = {key: widgets.Label(f"{key}: '{value}',") for key, value in a_session_tuple.items()}

    # Horizontally stack the label widgets
    # header_hbox = widgets.HBox(list(header_label_widgets.values()), layout=widgets.Layout(min_width='400px', min_height='50px', width='auto', height='auto')) # , overflow='auto'
    
    # Define a layout that enables wrapping
    box_layout = widgets.Layout(
        display='flex',
        flex_flow='row wrap',
        align_items='stretch',
        width='100%'
    )

    # Create a Box with the custom layout
    header_hbox = widgets.Box(list(header_label_widgets.values()), layout=box_layout)


    # Function to update the values in the labels
    def update_header_labels_fn(new_values):
        """ captures: label_widgets"""
        for key, value in new_values.items():
            header_label_widgets[key].value = f"{key}: {value}"
            
    # Display the widget
    return header_hbox, header_label_widgets, update_header_labels_fn


# Example SessionTuple
empty_session_tuple = {
    'format_name': '',
    'animal': '',
    'exper_name': '',
    'session_name': '',
    'session_descriptor_string': '',
    'id': '<Selection Not Found>',
    'hostname': '',
    'creation_time': '',
    'running_time': '',
    'ping_time': '',
    'monitoring_time': '',
    'size': '',
    'tags': '',
    'entrypoint': ''
}

header_hbox, header_label_widgets, update_header_labels_fn = build_session_tuple_header_widget(a_session_tuple=empty_session_tuple)

# Create Textarea widget with a defined width
textarea = widgets.Textarea(value='<No Selection>',
    disabled=True,  # Make it read-only
    style={'font_size': '10px'},  # Smaller font size
    layout=widgets.Layout(flex='1', width='650px', min_height='650px',
                        height='850px',
                        ))

content_view_layout = widgets.VBox([header_hbox, textarea], layout=widgets.Layout(min_width='500px', min_height='200px', width='auto', height='auto')) # INPUTS: header_hbox

# Layout widgets side by side with proper spacing
layout = widgets.HBox([jupyter_tree_widget.tree, content_view_layout], layout=widgets.Layout(min_width='500px', min_height='100px', width='auto', height='auto'))


def _on_tree_node_selection_changed(selected_node, selected_context):
    """ 
    captures: context_indexed_run_logs, textarea, most_recent_runs_context_indexed_run_extra_data, empty_session_tuple, update_header_labels_fn
    """
    print(f'_on_tree_node_selection_changed(selected_node: {selected_node}, selected_context: {selected_context})') # Selected context: {'format_name': 'kdiba', 'animal': 'pin01', 'exper_name': 'one', 'session_name': 'fet11-01_12-58-54'}
    #_on_tree_node_selection_changed(selected_node: Node(name='fet11-01_12-58-54', selected=True), selected_context: {'format_name': 'kdiba', 'animal': 'pin01', 'exper_name': 'one', 'session_name': 'fet11-01_12-58-54'})
    if isinstance(selected_context, dict):
        selected_context = IdentifyingContext(**selected_context)
    
    curr_context_extra_data_tuple = most_recent_runs_context_indexed_run_extra_data.get(selected_context, empty_session_tuple)
    update_header_labels_fn(curr_context_extra_data_tuple)
    curr_context_run_log: str = context_indexed_run_logs.get(selected_context, '<Context Not Found>')
    textarea.value = curr_context_run_log

jupyter_tree_widget.on_selection_changed_callback = [_on_tree_node_selection_changed]
# Display the layout
layout


In [ ]:


text_area = widgets.Textarea(
    value='<No Selection>',
    disabled=True,  # Make it read-only
    layout=widgets.Layout(width='100%', height='400px')  # Scrollable area
)

combined_widget = widgets.HBox([jupyter_tree_widget.tree, text_area])
combined_widget
# log_viewer = create_log_viewer(logs=run_logs)
# display(log_viewer)


In [ ]:
parsed_structure = run.get_parsed_structure()
parsed_structure

In [ ]:
import regions
(parsed_structure['monitoring'])


In [ ]:
for run_id, a_log in run_logs.items():
    print("# ==================================================================================================================== #")
    print(f"# run_id: {run_id}                                                                                                     #")
    print("# ==================================================================================================================== #")    
    # print(f'run_id: {run_id} =================')
    # print(f"# run_id: {run_id:<100} #")
    print(a_log)
    print("# END LOG ____________________________________________________________________________________________________________ #")
    


Calling `run.print_structure()` produces the following output:
```python
'animal': String
'exper_name': String
'format_name': String
'monitoring':
    '5f739afe':
        'hostname': String
        'pid': String
        'tid': String
    'be28f54f':
        'cpu': FloatSeries
        'hostname': String
        'memory': FloatSeries
        'pid': String
        'stderr': StringSeries
        'stdout': StringSeries
        'tid': String
'outputs':
    'figures':
        'display_fn_name:BatchPhoJonathanReplayFRC':
            'plot_result_set:long_only':
                'aclus:(16,19,6,31,10)': File
                'aclus:(6,10,16,19,31)': File
            'plot_result_set:shared':
                'page:1of2':
                    'aclus:(26,20,29,18,3,4,7,32,13,11,12,2,30,15,9,28,22,14,21,25)': File
                'page:1of3':
                    'aclus:(2,3,4,5,7,8,9,11,12,13)': File
                'page:2of2':
                    'aclus:(5,8,17,23,27)': File
                'page:2of3':
                    'aclus:(14,15,17,18,20,21,22,23,25,26)': File
                'page:3of3':
                    'aclus:(27,28,29,30,32)': File
            'plot_result_set:short_only':
                'aclus:(24)': File
        'display_fn_name:_display_grid_bin_bounds_validation': File
        'display_fn_name:bidir_track_remap':
            'subplot_name:Track Remapping': File
        'display_fn_name:display_long_short_laps': File
        'display_fn_name:display_short_long_pf1D_comparison':
            'track:long': File
            'track:short': File
        'display_fn_name:plot_all_epoch_bins_marginal_predictions':
            'subplot_name:Laps all_epoch_binned Marginals': File
            'subplot_name:Ripple all_epoch_binned Marginals': File
        'display_fn_name:plot_expected_vs_observed':
            'x_variable:epoch_idx':
                'variable:obs_exp_diff_ptp': File
        'display_fn_name:plot_histograms':
            'subplot_name:laps_result_tuple': File
            'subplot_name:ripple_result_tuple': File
        'display_fn_name:plot_quantile_diffs':
            'subplot_name:BestDir': File
        'display_fn_name:plot_rank_order_epoch_inst_fr_result_tuples':
            'subplot_name:Lap':
                'subplotsubplot_name:raw': File
                'subplotsubplot_name:z_score_diff': File
            'subplot_name:Ripple':
                'subplotsubplot_name:raw': File
                'subplotsubplot_name:z_score_diff': File
        'display_fn_name:plot_rank_order_histograms':
            'subplot_name:Ripple Most-likely Spearman Rho': File
            'subplot_name:Ripple Most-likely Z-scores': File
            'subplot_name:Ripple Z-scores': File
            'subplot_name:Ripple real correlations': File
        'display_fn_name:running_and_replay_speeds_over_time': File
        'filter_name:maze1_any':
            'lap_dir:any':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
                'display_fn_name:plot_single_track_firing_rate_compare': File
        'filter_name:maze1_even':
            'lap_dir:even':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze1_odd':
            'lap_dir:odd':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze2_any':
            'lap_dir:any':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
                'display_fn_name:plot_single_track_firing_rate_compare': File
        'filter_name:maze2_even':
            'lap_dir:even':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze2_odd':
            'lap_dir:odd':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze_any':
            'lap_dir:any':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze_even':
            'lap_dir:even':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze_odd':
            'lap_dir:odd':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'fn_name:long_short_firing_rate_indicies':
            'display_fn_name:display_long_short_laps': File
'session_descriptor_string': String
'session_name': String
'source_code':
    'diff': File
    'entrypoint': String
'sys':
    'creation_time': Datetime
    'description': String
    'failed': Boolean
    'group_tags': StringSet
    'hostname': String
    'id': String
    'modification_time': Datetime
    'monitoring_time': Integer
    'name': String
    'owner': String
    'ping_time': Datetime
    'running_time': Float
    'size': Float
    'state': RunState
    'tags': StringSet
    'trashed': Boolean
```
How can I capture this printed output (printed to stdout I think) and then parse it into a tree-like structure made of nested-dictionaries to be able to figure out the available structure?

In [ ]:
# Fetch all experiments
experiments = neptuner.project.fetch_experiments_table().to_pandas()
experiments

In [ ]:
# Initialize the neptune client
# neptune.init(project=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])

# project = neptune.get_project()
# project

In [ ]:


if neptuner.run is None:
    neptuner.run = AutoValueConvertingNeptuneRun(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath])
    params = {"TODAY_DAY_DATE": TODAY_DAY_DATE, "run_workstation": "Apogee"}
    neptuner.run["parameters"] = params
    neptuner.outputs = neptuner.run['outputs']
    neptuner.figures = neptuner.outputs['figures']

neptuner_run: AutoValueConvertingNeptuneRun = neptuner.run

# run = neptune.init_run(source_files=["**/*.dvc"])

# # Pre-execution dataframe view:
# run["dataset/global_batch_run_progress_df"].upload(File.as_html(global_batch_run.to_dataframe(expand_context=True, good_only=False))) # "path/to/test_preds.csv"



known_bad_sessions = [IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44')]
known_bad_session_strs = [str(v.get_description()) for v in known_bad_sessions]
known_bad_session_strs

In [ ]:
from pyphocorehelpers.gui.Qt.testLogFileViewer import LogViewerWidget

# Call the function to build and display the interactive session display
# interactive_layout = Neptuner.build_interactive_session_run_logs_widget(context_indexed_run_logs, most_recent_runs_session_descriptor_string_to_context_map, most_recent_runs_context_indexed_run_extra_data)
# display(interactive_layout)

# Instantiate the widget
viewer = LogViewerWidget(context_indexed_run_logs, most_recent_runs_session_descriptor_string_to_context_map, most_recent_runs_context_indexed_run_extra_data)

# Set window title or other properties if needed
viewer.setWindowTitle('Session Log Viewer')

# Show the widget
viewer.show()